We'll start by using the [markovify](https://github.com/jsvine/markovify/) library to make some social-media-sized utterances in the style of Jane Austen.  This will be the basis for generating a synthetic social media stream.

In [21]:
import markovify
import codecs

with codecs.open("austen.txt", "r", "cp1252") as f:
    text = f.read()

austen_model = markovify.Text(text)

for i in range(10):
    print(austen_model.make_short_sentence(200))

Her sorrow, her disappointment, her deep regret, when I take on my guard.
Men are very proud of him.
Some people call him earlier home.
There is not five minutes before she was so plain and her thoughts or fix her wedding-day--and Mr. Elton sauntering about.
The former she was sometimes quite provoked, but then she began her relation.
The anxieties of common occurrences with all her notions of what she could do nothing at all.
She had already mentioned their wishes and affection equally dictated, and I understand it now, and very kind reception of me which I last night to indulge them.
Elinor could hardly separate the parts, so as not unlikely that some pains are taken to prevent her from Barton.
They will have nothing clever to say--not one of the agitation which she had been made amiable himself; for he vowed he would have thought necessary.
Nothing could be three.


We'll use spaCy to identify entities (mostly proper nouns and noun phrases) in these synthetic status updates and turn them into hashtags:

In [23]:
import spacy
nlp = spacy.load('en')

def make_sentence(model, length=200):
    return model.make_short_sentence(length)
    
def hashtagify(sentence):
    doc = nlp(sentence)
    for ent in doc.ents:
        sentence = sentence.replace(str(ent), "#%s" % str(ent).replace(" ", ""))
    return sentence

for i in range(5):
    sentence = make_sentence(austen_model)
    print(sentence)
    print(hashtagify(sentence))

Chapter 11 The morrow produced no abatement of Mrs. Dashwood it was not hopeless.
#Chapter11 The #morrow produced no abatement of Mrs. #Dashwood it was not hopeless.
Mr. Darcy, with a very good-natured fellow: while Lady Middleton could no longer what it is!
Mr. #Darcy, with a very good-natured fellow: while #LadyMiddleton could no longer what it is!
Kitty, to her his hand.
#Kitty, to her his hand.
If she could not be a different kind.
If she could not be a different kind.
It is disgraceful to us in mind of Isabella.
It is disgraceful to us in mind of #Isabella.


We'll now train two Markov models on positive and negative product reviews (taken from the [public-domain Amazon fine foods reviews dataset on Kaggle](https://www.kaggle.com/snap/amazon-fine-food-reviews/)).  We'll incorporate the results of these models into our synthetic social media stream.

In [47]:
import gzip

def train_markov_gz(fn):
    """ trains a Markov model on gzipped text data """
    with gzip.open(fn, "rt", encoding="utf-8") as f:
        text = f.read()
    return markovify.Text(text)

negative_model = train_markov_gz("reviews-1.txt.gz")
positive_model = train_markov_gz("reviews-5-100k.txt.gz")

In [48]:
make_sentence(negative_model)

'This new formula other day -- GROSS!'

In [49]:
make_sentence(positive_model)

'She has trouble digesting kibble.'